In [40]:
# Cell #: 1 Library Imports
import pandas as pd
import networkx as nx
import uunet.multinet as ml
import itertools
import numpy as np  
from math import log

In [41]:
#Cell #: 2 File Paths
# File paths for the Excel sheets
file_paths = [
             "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Layer_1_70percent.xlsx", 
             "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Layer_2_70percent.xlsx", 
             "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Layer_3_70percent.xlsx", 
             "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Layer_4_70percent.xlsx", 
             "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Layer_5_70percent.xlsx"
             ]


In [ ]:
# Cell #: 3 Constructing the Multiplex Network

print("Initializing multiplex network...")
multiplex_net = ml.empty()

# A variable to keep track of the layer index
layer_index = 1

# Iterate over each Excel file and add its sheets as layers to the multiplex network
for file_path in file_paths:
    print(f"\nProcessing file: {file_path}")
    xls = pd.ExcelFile(file_path)

    for sheet_name in xls.sheet_names:
        unique_layer_name = f"Layer_{layer_index}"
        print(f"\nAdding layer: {unique_layer_name}")
        df = pd.read_excel(xls, sheet_name)

        # Debug: Print a sample of the DataFrame
        print(f"DataFrame sample from {unique_layer_name}:\n", df.head())

        vertices = {'actor': list(set(df['From_Node']).union(df['To_Node'])), 
                    'layer': [unique_layer_name] * len(set(df['From_Node']).union(df['To_Node']))}
        print(f"Adding vertices to layer {unique_layer_name}: {vertices['actor']}")

        ml.add_vertices(multiplex_net, vertices)

        edges = {
            'from_actor': df['From_Node'].tolist(),
            'from_layer': [unique_layer_name] * len(df),
            'to_actor': df['To_Node'].tolist(),
            'to_layer': [unique_layer_name] * len(df)
        }
        print(f"Adding {len(df)} edges to layer {unique_layer_name}")
        print(f"Sample edges for layer {unique_layer_name}: {edges['from_actor'][:5]} -> {edges['to_actor'][:5]}")

        ml.add_edges(multiplex_net, edges)

        layer_index += 1

print("\nNetwork construction completed.")
print(f"Total layers in network: {len(ml.layers(multiplex_net))}")


In [43]:
# Cell #: 4 Finding the Exclusive Neighbors
# Retrieve the exclusive neighbors of all actors in all Layers
all_layers = ml.layers(multiplex_net)

exclusive_neighbors = {}

for layer in all_layers:
    print(f"\nFinding exclusive neighbors for layer {layer}")

    # Retrieve all vertices in the current layer
    vertices_info = ml.vertices(multiplex_net, [layer])
    actors = vertices_info['actor']  # List of actor IDs in the layer

    # Prepare to store exclusive neighbors for the current layer
    exclusive_neighbors[layer] = {}

    # Find exclusive neighbors for each actor in the layer
    for actor_id in actors:
        # Retrieve exclusive neighbors for the actor
        neighbors = ml.xneighbors(multiplex_net, actor_id, layers=[layer], mode='all')
        exclusive_neighbors[layer][actor_id] = neighbors

# Commented out file saving part
# file_path = '/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/exclusive_neighbors_Indivisual_layers70.txt'
# with open(file_path, 'w') as file:
#     for layer, neighbors_dict in exclusive_neighbors.items():
#         file.write(f"Layer {layer}:\n")
#         for actor_id, neighbors in neighbors_dict.items():
#             if neighbors:  # Only write if the actor has exclusive neighbors
#                 file.write(f"Actor {actor_id}: Exclusive neighbors: {neighbors}\n")
#         file.write("\n")

# Display the exclusive neighbors data
for layer, neighbors_dict in exclusive_neighbors.items():
    print(f"Layer {layer}:")
    for actor_id, neighbors in neighbors_dict.items():
        if neighbors:  # Only display if the actor has exclusive neighbors
            print(f"Actor {actor_id}: Exclusive neighbors: {neighbors}")
    print("\n")



Finding exclusive neighbors for layer Layer_2

Finding exclusive neighbors for layer Layer_1

Finding exclusive neighbors for layer Layer_4

Finding exclusive neighbors for layer Layer_5

Finding exclusive neighbors for layer Layer_3
Layer Layer_2:
Actor 76: Exclusive neighbors: {'78', '79'}
Actor 78: Exclusive neighbors: {'76'}
Actor 127: Exclusive neighbors: {'117'}
Actor 79: Exclusive neighbors: {'76'}
Actor 117: Exclusive neighbors: {'127'}


Layer Layer_1:
Actor 6: Exclusive neighbors: {'36'}
Actor 36: Exclusive neighbors: {'6'}
Actor 175: Exclusive neighbors: {'191'}
Actor 191: Exclusive neighbors: {'175', '202'}
Actor 202: Exclusive neighbors: {'191'}


Layer Layer_4:


Layer Layer_5:
Actor 130: Exclusive neighbors: {'141'}
Actor 141: Exclusive neighbors: {'130'}


Layer Layer_3:




In [44]:
# Cell #: 5 Link Prediction : Through Modified Jaccard Coefficient
# Convert the multiplex network into NetworkX graphs for each layer
nx_graphs = ml.to_nx_dict(multiplex_net)

# Define the modified Jaccard coefficient function
def modified_jaccard_coefficient(G, exclusive_neighbors):
    def predict(u, v):
        neighbors_u = exclusive_neighbors.get(u, set())
        neighbors_v = exclusive_neighbors.get(v, set())
        union_size = len(neighbors_u | neighbors_v)
        if union_size == 0:
            return 0
        intersection_size = len(neighbors_u & neighbors_v)
        return intersection_size / union_size

    return ((u, v, predict(u, v)) for u, v in nx.non_edges(G))

# Iterate over each layer and apply the modified Jaccard coefficient function
for layer, exclusive_neighbors in exclusive_neighbors.items():
    if exclusive_neighbors:  # Check if the layer has exclusive neighbors
        G = nx_graphs[layer]
        link_predictions = list(modified_jaccard_coefficient(G, exclusive_neighbors))

        # Print link predictions for the current layer
        print(f"Link predictions for {layer}:")
        for u, v, score in link_predictions:
            if score > 0:  # Optionally filter out zero scores
                print(f"{u} - {v}: {score}")
    else:
        print(f"No exclusive neighbors for {layer}, skipping link prediction.")


Link predictions for Layer_2:
79 - 78: 1.0
Link predictions for Layer_1:
175 - 202: 1.0
Link predictions for Layer_4:
Link predictions for Layer_5:
Link predictions for Layer_3:


In [ ]:
# Cell #: 6 Link Prediction : Through Modified Adamic-Adar Index


# Define the file path and file name
file_path = "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Adamic-Adar-Score_Step01.txt"

# Assuming nx_graphs is a dictionary containing NetworkX graphs for each layer

# Exclusive neighbors data for Layer_1
exclusive_neighbors_layer_1 = {
    '6': {'11', '2', '36'},
    '11': {'6'},
    '13': {'30'},
    '2': {'6'},
    '23': {'34'},
    '30': {'13'},
    '34': {'23'},
    '36': {'6'},
    '160': {'176', '174'},
    '174': {'160'},
    '175': {'191'},
    '176': {'160'},
    '191': {'175'},
    '202': {'211'},
    '211': {'212', '202'},
    '212': {'211'}
}

# Exclusive neighbors data for Layer_5
exclusive_neighbors_layer_5 = {
    '26': {'37'},
    '37': {'11', '26', '33'},
    '48': {'28'},
    '19': {'16', '21'},
    '11': {'22', '37'},
    '21': {'19'},
    '22': {'11'},
    '28': {'48'},
    '33': {'37'},
    '127': {'139'},
    '139': {'127'},
    '16': {'19'}
}

# Define the modified Adamic-Adar index function
def modified_adamic_adar_index(G, exclusive_neighbors):
    def predict(u, v):
        common_neighbors = exclusive_neighbors.get(u, set()) & exclusive_neighbors.get(v, set())
        return sum(1 / log(len(exclusive_neighbors.get(w, set()))) for w in common_neighbors)

    return ((u, v, predict(u, v)) for u, v in nx.non_edges(G))

# Apply the modified Adamic-Adar index function to Layer_1
G_layer_1 = nx_graphs['Layer_1']
adamic_adar_predictions_layer_1 = list(modified_adamic_adar_index(G_layer_1, exclusive_neighbors_layer_1))

# Print Adamic-Adar predictions for Layer_1
print("Adamic-Adar predictions for Layer_1:")
for u, v, score in adamic_adar_predictions_layer_1:
    if score > 0:  # Optionally filter out zero scores
        print(f"{u} - {v}: {score}")

# Apply the modified Adamic-Adar index function to Layer_5
G_layer_5 = nx_graphs['Layer_5']
adamic_adar_predictions_layer_5 = list(modified_adamic_adar_index(G_layer_5, exclusive_neighbors_layer_5))

# Print Adamic-Adar predictions for Layer_5
print("Adamic-Adar predictions for Layer_5:")
for u, v, score in adamic_adar_predictions_layer_5:
    if score > 0:  # Optionally filter out zero scores
        print(f"{u} - {v}: {score}")
# Save the content to the file
with open(file_path, 'w') as file:
    file.write(content)

print(f"Results saved to {file_path}")



In [ ]:
# Cell #: Save the results to a file Jaccard Coefficient
# Define the file path and file name
file_path = "/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Jaccard_Score_Step01.txt"

# Prepare the content to be written to the file
content = "Link predictions for Layer_1:\n"
for u, v, score in link_predictions_layer_1:
    if score > 0:  # Optionally filter out zero scores
        content += f"{u} - {v}: {score}\n"

content += "\nLink predictions for Layer_5:\n"
for u, v, score in link_predictions_layer_5:
    if score > 0:  # Optionally filter out zero scores
        content += f"{u} - {v}: {score}\n"

# Save the content to the file
with open(file_path, 'w') as file:
    file.write(content)

print(f"Results saved to {file_path}")


In [15]:

#### Debuging Cell ####
# Extract all layers from the multiplex network
all_layers = ml.layers(multiplex_net)

# Check if Layer 1 is present in the network
if 'Layer_1' in all_layers:
    # Retrieve all actors (nodes) in Layer 1
    actors_layer_1 = ml.actors(multiplex_net, layers=['Layer_1'])

    # Retrieve all edges (connections) in Layer 1
    edges_layer_1 = ml.edges(multiplex_net, layers1=['Layer_1'], layers2=['Layer_1'])

    # Format the edges data correctly using the provided keys
    edges_data = list(zip(edges_layer_1['from_actor'], edges_layer_1['to_actor']))

    # Print the formatted edges data
    print("Edges in Layer 1:", edges_data)
else:
    print("Layer_1 not found in the network.")


Edges in Layer 1: [('38', '66'), ('10', '56'), ('14', '3'), ('19', '67'), ('51', '73'), ('31', '50'), ('3', '52'), ('1', '47'), ('6', '2'), ('15', '22'), ('58', '59'), ('58', '74'), ('4', '42'), ('20', '28'), ('42', '25'), ('43', '17'), ('17', '75'), ('3', '34'), ('13', '30'), ('34', '23'), ('15', '24'), ('39', '28'), ('43', '15'), ('21', '23'), ('8', '35'), ('11', '1'), ('6', '11'), ('52', '72'), ('24', '25'), ('4', '22'), ('14', '71'), ('11', '17'), ('57', '58'), ('33', '35'), ('5', '33'), ('11', '62'), ('28', '62'), ('39', '17'), ('9', '57'), ('5', '35'), ('9', '34'), ('7', '33'), ('19', '20'), ('164', '178'), ('7', '11'), ('27', '11'), ('32', '21'), ('54', '11'), ('53', '11'), ('6', '36'), ('5', '32'), ('68', '69'), ('123', '115'), ('123', '114'), ('118', '110'), ('110', '119'), ('119', '128'), ('137', '125'), ('140', '137'), ('128', '140'), ('125', '124'), ('135', '147'), ('124', '135'), ('183', '181'), ('147', '157'), ('202', '189'), ('168', '183'), ('189', '161'), ('160', '149')

In [ ]:
help(ml.xneighbors  )

In [ ]:
dir(ml)

In [38]:
# Perecentage of edges to keep Code

import pandas as pd

def keep_top_70_percent_edges(file_path):
    # Load the Excel file
    data = pd.read_excel(file_path)

    # Sort the edges based on 'Flow' in descending order
    sorted_data = data.sort_values(by='Flow', ascending=False)

    # Calculate the number of edges to keep (70% of the total edges)
    number_of_edges_to_keep = int(len(sorted_data) * 0.7)

    # Select the top 70% edges
    top_70_percent_edges = sorted_data.head(number_of_edges_to_keep)

    return top_70_percent_edges

# Example usage for a single file
file_path = '/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/Case_5.xlsx'  # Replace with your file path
top_70_percent_edges = keep_top_70_percent_edges(file_path)

# You can then view or save the result as needed
print(top_70_percent_edges)
top_70_percent_edges.to_excel('/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/Data/Updated Data/top_70_percent_edges_sheet5.xlsx')  # To save the result


     From_Node  From_Layer  To_Node  To_Layer         Flow
88          68           1       69         1  3223.971139
87          11           1       37         1  2808.415135
86           5           1       32         1  2359.085320
85          11           1       61         1  2312.460471
84          33           1       37         1  2116.959750
..         ...         ...      ...       ...          ...
126        175           2      160         2    21.977866
199        205           2      192         2    21.818687
198        192           2      175         2    21.774148
127        192           2      175         2    21.616584
197        205           2      192         2    21.572046

[186 rows x 5 columns]
